# Using XGBoost in pipelines
Take your XGBoost skills to the next level by incorporating your models into two end-to-end machine learning pipelines. You'll learn how to tune the most important XGBoost hyperparameters efficiently within a pipeline, and get an introduction to some more advanced preprocessing techniques. This is the Summary of lecture "Extreme Gradient Boosting with XGBoost", via datacamp.

## Review of pipelines using sklearn
  
Pipeline review
- Takes a list of 2-tuples (name, pipeline_step) as input
- Tuples can contain any arbitrary scikit-learn compatible estimator or transformer object
- Pipeline implements fit/predict methods
- Can be used as input estimator into grid/randomized search and `cross_val_score` methods
  
Preprocessing I: LabelEncoder and OneHotEncoder  
The first approach involves using the `LabelEncoder` and `OneHotEncoder` classes of scikit-learn’s preprocessing submodule one after the other. The problem with this 2-step method, however, is that it cannot currently be done within a pipeline.  
- `LabelEncoder` simply converts a categorical column of strings into integers that map onto those strings.  
- `OneHotEncoder` takes a column of integers that are treated as categorical values, and encodes them as dummy variables.
  
Preprocessing II: DictVectorize  
The second approach, which involves using a dict-vectorizer, can accomplish both steps in one line of code.The `DictVectorizer` is a class found in scikit-learn’s feature extraction submodule, and is **traditionally used in text processing pipelines** by converting lists of feature mappings into vectors. Using pandas DataFrames, we don’t initially have such a list, however, if we explicitly convert a DataFrame into a list of dictionary entries, then we have exactly what we need. 

### Encoding categorical columns I - LabelEncoder
  
Now that you've seen what will need to be done to get the housing data ready for XGBoost, let's go through the process step-by-step.
  
First, you will need to fill in missing values - as you saw previously, the column LotFrontage has many missing values. Then, you will need to encode any categorical columns in the dataset using one-hot encoding so that they are encoded numerically.
  
The data has five categorical columns: MSZoning, PavedDrive, Neighborhood, BldgType, and HouseStyle. Scikit-learn has a `LabelEncoder` function that converts the values in each categorical column into integers. You'll practice using this here.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb


# Load data
df = pd.read_csv('../_datasets/ames_unprocessed_data.csv')
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,...,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageArea,PavedDrive,SalePrice
0,60,RL,65.0,8450,CollgCr,1Fam,2Story,7,5,2003,...,1710,1,0,2,1,3,0,548,Y,208500
1,20,RL,80.0,9600,Veenker,1Fam,1Story,6,8,1976,...,1262,0,1,2,0,3,1,460,Y,181500
2,60,RL,68.0,11250,CollgCr,1Fam,2Story,7,5,2001,...,1786,1,0,2,1,3,1,608,Y,223500
3,70,RL,60.0,9550,Crawfor,1Fam,2Story,7,5,1915,...,1717,1,0,1,0,3,1,642,Y,140000
4,60,RL,84.0,14260,NoRidge,1Fam,2Story,8,5,2000,...,2198,1,0,2,1,4,1,836,Y,250000


In [2]:
from sklearn.preprocessing import LabelEncoder


# Filling missing values with 0
df.LotFrontage = df.LotFrontage.fillna(0)

# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == 'object')

# Get list of categorical columns by name
categorical_columns = df.columns[categorical_mask].tolist()

# Print the head of the categorical columns
print(df[categorical_columns].head())

# Create LabelEncoder object
le = LabelEncoder()

# Apply LabelEncode to categorical columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))

# Print the head of the LabelEncode categorical
print(df[categorical_columns].head())

  MSZoning Neighborhood BldgType HouseStyle PavedDrive
0       RL      CollgCr     1Fam     2Story          Y
1       RL      Veenker     1Fam     1Story          Y
2       RL      CollgCr     1Fam     2Story          Y
3       RL      Crawfor     1Fam     2Story          Y
4       RL      NoRidge     1Fam     2Story          Y
   MSZoning  Neighborhood  BldgType  HouseStyle  PavedDrive
0         3             5         0           5           2
1         3            24         0           2           2
2         3             5         0           5           2
3         3             6         0           5           2
4         3            15         0           5           2


### Encoding categorical columns II - OneHotEncoder
  
Okay - so you have your categorical columns encoded numerically. Can you now move onto using pipelines and XGBoost? Not yet! In the categorical columns of this dataset, there is no natural ordering between the entries. As an example: Using `LabelEncoder`, the CollgCr Neighborhood was encoded as 5, while the Veenker Neighborhood was encoded as 24, and Crawfor as 6. Is Veenker "greater" than Crawfor and CollgCr? No - and allowing the model to assume this natural ordering may result in poor performance.  
  
As a result, there is another step needed: You have to apply a one-hot encoding to create binary, or "dummy" variables. You can do this using scikit-learn's `OneHotEncoder`.
  
> Warning: Instead of using `LabelEncoder`, use `make_column_transformer`
  
The `make_column_transformer()` function is a utility function that creates a transformer for pre-processing columns of heterogeneous data, which can include both numerical and categorical data, in a way that is compatible with the scikit-learn pipeline architecture.

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# Load data
df = pd.read_csv('../_datasets/ames_unprocessed_data.csv')

# Fill missing values with 0
df.LotFrontage = df.LotFrontage.fillna(0)

# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == 'object')

# Get list of categorical columns names
categorical_columns = df.columns[categorical_mask].tolist()

# Generate a unique list containing each of the categorical columns and their unique values
unique_list = [df[c].unique().tolist() for c in categorical_columns]

# Create OneHotEncoder
ohe = OneHotEncoder(categories= unique_list)

# Create preprocess object for one-hot encoding
# We want the non-categories to pass, so we take the categorical_mask, get only columns that are not 
# catagorical (False), make them an index, then turn them to a list.
preprocess = make_column_transformer(
    (ohe, categorical_columns),
    ('passthrough', categorical_mask[~categorical_mask].index.to_list())
)

# Apply OneHotEncoder to categorical columns - output is no longer a dataframe
df_encoded = preprocess.fit_transform(df)

# Print first 5 rows of the resulting dataset - again, this is no longer a dataframe
print(df_encoded[:5, :])

# Print the shape for the original dataframe
print(df.shape)

# Print the shape of the transformed array
print(df_encoded.shape)


[[1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 6.000e+01 6.500e+01 8.450e+03
  7.000e+00 5.000e+00 2.003e+03 0.000e+00 1.710e+03 1.000e+00 0.000e+00
  2.000e+00 1.000e+00 3.000e+00 0.000e+00 5.480e+02 2.085e+05]
 [1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

### Encoding categorical columns III: DictVectorizer
  
Alright, one final trick before you dive into pipelines. The two step process you just went through - `LabelEncoder` followed by `OneHotEncoder` - can be simplified by using a `DictVectorizer`.
  
Using a `DictVectorizer` on a DataFrame that has been converted to a dictionary allows you to get label encoding as well as one-hot encoding in one go.
  
Your task is to work through this strategy in this exercise!

In [4]:
from sklearn.feature_extraction import DictVectorizer


# Convert df into a dictionary
df_dict = df.to_dict('records')

# Create the DictVectorizer object
dv = DictVectorizer(sparse= False)

# Apply dv df
df_encoded2 = dv.fit_transform(df_dict)

# Print the resulting first five rows
print(df_encoded2[:5, :])

# Print the vocaulary
print(dv.vocabulary_)

[[3.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 2.000e+00 5.480e+02 1.710e+03 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  8.450e+03 6.500e+01 6.000e+01 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 5.000e+00 7.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 2.085e+05 2.003e+03]
 [3.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.000e+00 1.000e+00 2.000e+00 4.600e+02 1.262e+03 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  9.600e+03 8.000e+01 2.000e+01 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

Besides simplifying the process into one step, `DictVectorizer` has useful attributes such as `vocabulary_` which maps the names of the features to their indices. With the data preprocessed, it's time to move onto pipelines!

### Preprocessing within a pipeline
  
Now that you've seen what steps need to be taken individually to properly process the Ames housing data, let's use the much cleaner and more succinct `DictVectorizer` approach and put it alongside an `XGBoostRegressor` inside of a scikit-learn pipeline.

In [5]:
from sklearn.pipeline import Pipeline

# Load data
df = pd.read_csv('../_datasets/ames_unprocessed_data.csv')

# X/y split
X, y = df.iloc[:, :-1], df.iloc[:, -1]

# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps
steps = [
    ('ohe_onestep', DictVectorizer(sparse= False)),
    ('xgb_model', xgb.XGBRegressor())
]

# Create the pipeline
xgb_pipeline = Pipeline(steps)

# Fit the pipeline
xgb_pipeline.fit(X.to_dict('records'), y)

Pipeline(steps=[('ohe_onestep', DictVectorizer(sparse=False)),
                ('xgb_model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

## Incorporating XGBoost into pipelines
  
Additional components introduced for pipelines  
  
`sklearn_pandas`:
- `DataFrameMapper` - Interoperability between pandas and scikit-learn
- `CategoricalImputer` - Allow for imputation of categorical variables before conversion to integers
  
`sklearn.preprocessing`:
- `Imputer` - Native imputation of numerical columns in scikit-learn
  
`sklearn.pipeline`:
- `FeatureUnion` - combine multiple pipelines of features into a single pipeline of features. For example, if we needed one set of preprocessing steps preformed on the categorical features of a dataset and a distinct set of preprocessing steps on the numeric features found in a dataset.
  

### Cross-validating your XGBoost model
  
In this exercise, you'll go one step further by using the pipeline you've created to preprocess and cross-validate your model.

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


# Load data
df = pd.read_csv('../_datasets/ames_unprocessed_data.csv')

# X/y split
X, y = df.iloc[:, :-1], df.iloc[:, -1]

# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps
steps = [
    ('ohe_onestep', DictVectorizer(sparse= False)),
    ('xgb_model', xgb.XGBRegressor(max_depth= 2, objective= 'reg:squarederror'))
]

# Create the pipeline
xgb_pipeline = Pipeline(steps)

# Cross-validate the model
cross_val_scores = cross_val_score(xgb_pipeline, X.to_dict('records'), y, scoring= 'neg_mean_squared_error', cv= 10)

# Final RMSE
final_rmse = np.mean(np.sqrt(np.abs(cross_val_scores)))

# Print the 10-fold RMSE
print('10-fold RMSE: {}'.format(final_rmse))

10-fold RMSE: 27683.04157118635


### Kidney disease case study I - Categorical Imputer
  
You'll now continue your exploration of using pipelines with a dataset that requires significantly more wrangling. The chronic kidney disease dataset contains both categorical and numeric features, but contains lots of missing values. The goal here is to predict who has chronic kidney disease given various blood indicators as features.  
  
You'll be introduced to a new library, `sklearn_pandas`, that allows you to chain many more processing steps inside of a pipeline than are currently supported in scikit-learn. Specifically, you'll be able to impute missing categorical values directly using the `Categorical_Imputer()` class in `sklearn_pandas`, and the `DataFrameMapper()` class to apply any arbitrary sklearn-compatible transformer on DataFrame columns, where the resulting output can be either a NumPy array or DataFrame.  
  
We've also created a transformer called a `Dictifier` that encapsulates converting a DataFrame using `.to_dict("records")` without you having to do it explicitly (and so that it works in a pipeline). Finally, we've also provided the list of feature names in kidney_feature_names, the target name in kidney_target_name, the features in X, and the target in y.  
  
In this exercise, your task is to apply the `CategoricalImputer` to impute all of the categorical columns in the dataset. You can refer to how the numeric imputation mapper was created as a template. Notice the keyword arguments `input_df=True` and `df_out=True`? This is so that you can work with DataFrames instead of arrays. By default, the transformers are passed a numpy array of the selected columns as input, and as a result, the output of the DataFrame mapper is also an array. Scikit-learn transformers have historically been designed to work with numpy arrays, not pandas DataFrames, even though their basic indexing interfaces are similar.  
  
> WARNING: `CategoricalImputer` class has been removed from the sklearn_pandas library as of version 2.0.0.
> The recommended alternative is to use the `SimpleImputer` class from scikit-learn with the `strategy='most_frequent'` parameter to impute missing values in categorical columns with the most frequent value.

In [7]:
# pip install sklearn-pandas
from sklearn_pandas import DataFrameMapper
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd


X = pd.read_csv('../_datasets/chronic_kidney_X.csv')
y = pd.read_csv('../_datasets/chronic_kidney_y.csv').to_numpy().ravel()

# Check number of nulls in each of the feature columns
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()

# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

# Apply numeric imputer
numeric_imputation_mapper = DataFrameMapper(
    [([numeric_feature], SimpleImputer(strategy= 'median')) for numeric_feature in non_categorical_columns],
    input_df= True,
    df_out= True
)


# Apply categorical imputer
categorical_imputation_mapper = DataFrameMapper(
    [(category_feature, SimpleImputer(strategy= 'most_frequent')) for category_feature in categorical_columns],
    input_df= True,
    df_out= True
)

age        9
bp        12
sg        47
al        46
su        49
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wc       106
rc       131
rbc      152
pc        65
pcc        4
ba         4
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64


### Kidney disease case study II - Feature Union
  
Having separately imputed numeric as well as categorical columns, your task is now to use scikit-learn's `FeatureUnion()` to concatenate their results, which are contained in two separate transformer objects - `numeric_imputation_mapper`, and `categorical_imputation_mapper`, respectively. Just like with pipelines, you have to pass it a list of (string, transformer) tuples, where the first half of each tuple is the name of the transformer.  

In [8]:
from sklearn.pipeline import FeatureUnion


# Combine the numeric and categorical transformations
# Simple example of FeatureUnion
numeric_categorical_union = FeatureUnion([
    ('num_mapper', numeric_imputation_mapper),
    ('cat_mapper', categorical_imputation_mapper)
])

### Kidney disease case study III - Full pipeline
  
It's time to piece together all of the transforms along with an `XGBClassifier` to build the full pipeline!

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define transformers for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Define column transformer to apply different transformers to each column
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, non_categorical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Create full pipeline with preprocessing and classifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', xgb.XGBClassifier(max_depth=3))
])

# Preform cross-validation
cross_val_scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=3)

# Print average AUC
print('3-fold AUC: {}'.format(np.mean(cross_val_scores)))

3-fold AUC: 0.998237712755785


## Tuning XGBoost hyperparameters
### Bringing it all together
  
Alright, it's time to bring together everything you've learned so far! In this final exercise of the course, you will combine your work from the previous exercises into one end-to-end XGBoost pipeline to really cement your understanding of preprocessing and pipelines in XGBoost.
  
Your work from the previous 3 exercises, where you preprocessed the data and set up your pipeline, has been pre-loaded. Your job is to perform a randomized search and identify the best hyperparameters.

In [10]:
from sklearn.model_selection import RandomizedSearchCV


# Create the parameter grid
gbm_param_grid = {
    'clf__learning_rate' : np.arange(0.05, 1, 0.05),
    'clf__max_depth' : np.arange(3, 10, 1),
    'clf__n_estimators' : np.arange(50, 200, 50)
}

# Preform RandomizedSearchCV
randomized_roc_auc = RandomizedSearchCV(
    estimator= pipeline,
    param_distributions= gbm_param_grid,
    n_iter= 2,
    scoring= 'roc_auc',
    cv= 2,
    verbose= 1
)

# Fit the estimator
randomized_roc_auc.fit(X, y)

# Compute metrics
print('Score: {}'.format(randomized_roc_auc.best_score_))
print('Estimator: {}'.format(randomized_roc_auc.best_estimator_))

Fitting 2 folds for each of 2 candidates, totalling 4 fits
Score: 0.9980266666666666
Estimator: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['age', 'bp', 'sg', 'al',
                                                   'su', 'bgr', 'bu', 'sc',
                                                   'sod', 'pot', 'hemo', 'pcv',
                                                   'wc', 'rc']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
              

## Final Thoughts
  
Advanced Topic  
- Using XGBoost for ranking/recommandation problems (Netflix/Amazon problem)
- Using more sophisticated hyperparamter tuning strategies for tuning XGBoost model (Bayesian Optimization)
- Using XGBoost as part of an ensemble of other models for regression/classification

However, although we've covered quite a lot, we didn't cover some other topics that would advance your mastery of XGBoost. Specifically, we never looked into how to use XGBoost for ranking or recommendation problems, which can be done by modifying the loss function you use when constructing your model. We also didn't look into more advanced hyperparameter selection strategies. The most powerful strategy, called Bayesian optimization, has been used with lots of success, and entire companies have been created just for specifically using this method in tuning models (for example, the company sigopt does exactly this). It's a powerful method, but would take an entire other DataCamp course to teach properly! Finally, we haven't talked about ensembling XGBoost with other models. Although XGBoost is itself an ensemble method, nothing stops you from combining the predictions you get from an XGBoost model with other models, as this is usually a very powerful additional way to squeeze the last bit of juice from your data. Learning about all of these additional topics will help you become an even more powerful user of XGBoost. Now that you know your way around the package, there's no reason for you to stop learning how to get even more benefits out of it.